In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver # kinda of checkpointer (and it saves sessions/ state in ram (USE for temp but it real case we use different we will look after that))

In [2]:
load_dotenv()

True

In [3]:
llm = ChatGroq(model="llama-3.1-8b-instant")

In [4]:
class JokeState(TypedDict):
    topic: str
    joke: str
    explanation: str

In [5]:
def generate_joke(state: JokeState):

    prompt = f'generate a joke on the topic {state["topic"]}'
    response = llm.invoke(prompt).content

    return {'joke': response}

In [6]:
def generate_explanation(state: JokeState):

    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = llm.invoke(prompt).content

    return {'explanation': response}

In [7]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [10]:
config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({'topic':'bannana'}, config=config1)

{'topic': 'bannana',
 'joke': "Why did the banana go to the doctor?\n\nBecause he wasn't peeling well.",
 'explanation': 'The joke is a play on words, using a pun to create a humorous effect. The phrase "peeling well" has a double meaning here:\n\n1. In a literal sense, a banana has a peel, which is its outer layer. The joke is referencing the banana\'s physical appearance.\n\n2. In an idiomatic sense, "peeling well" sounds similar to "feeling well," which means being in good health. When someone is "peeling well," it implies that they are doing well or feeling good.\n\nThe joke is funny because it takes the common phrase "feeling well" and replaces it with a phrase that\'s related to a banana\'s physical characteristics, creating a clever and unexpected connection between the question and the answer. The punchline "he wasn\'t peeling well" is a clever play on words that uses the banana\'s unique characteristic to create a humorous effect.'}

In [11]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'bannana', 'joke': "Why did the banana go to the doctor?\n\nBecause he wasn't peeling well.", 'explanation': 'The joke is a play on words, using a pun to create a humorous effect. The phrase "peeling well" has a double meaning here:\n\n1. In a literal sense, a banana has a peel, which is its outer layer. The joke is referencing the banana\'s physical appearance.\n\n2. In an idiomatic sense, "peeling well" sounds similar to "feeling well," which means being in good health. When someone is "peeling well," it implies that they are doing well or feeling good.\n\nThe joke is funny because it takes the common phrase "feeling well" and replaces it with a phrase that\'s related to a banana\'s physical characteristics, creating a clever and unexpected connection between the question and the answer. The punchline "he wasn\'t peeling well" is a clever play on words that uses the banana\'s unique characteristic to create a humorous effect.'}, next=(), config={'config

In [13]:
workflow.get_state_history(config1)

<generator object Pregel.get_state_history at 0x0000026E98F9ADA0>